In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pandas as pd



In [2]:
import json

class Settings(dict):
    _instance = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls, *args, **kwargs)
        return cls._instance

    def __init__(self):
        self.load_settings()

    def load_settings(self):
        with open('settings.json', 'r') as file:
            settings = json.load(file)
            settings["prices"] = np.array(settings["prices"], dtype=np.float32)
            self.clear()
            self.update(settings)

    def update_settings(self):
        self.load_settings()

settings = Settings()
settings["prices"]



array([ 10.,  20.,  30.,  60., 100.], dtype=float32)

In [3]:
#generate a population with two binary features
def generate_population():
    f1 =  np.random.choice([0, 1], size=settings["sample_size"], p=[1 - settings["pf1"], settings["pf1"]])
    f2 =  np.random.choice([0, 1], size=settings["sample_size"], p=[1 - settings["pf2"], settings["pf2"]])
    return np.array(list(zip(f1, f2)))

In [4]:
#splits the sample into three groups based on the features
def classify(population):
    classifier = lambda x: 3 if x[0] == 1 else 2 if x[1] == 1 else 1
    return np.array([classifier(x) for x in population])

In [5]:
class Payment_characteristics_generator:
    _reasonable_price = {1: settings["c1_reasonable_price"], 2: settings["c2_reasonable_price"], 3: settings["c3_reasonable_price"]}
    _elasticity = {1: settings["c1_elasticity"], 2: settings["c2_elasticity"], 3: settings["c3_elasticity"]}
    _skepticism = {1: settings["c1_skepticism"], 2: settings["c2_skepticism"], 3: settings["c3_skepticism"]}
    _max_conversion_rate = {1: settings["c1_max_conversion_rate"], 2: settings["c2_max_conversion_rate"], 3: settings["c3_max_conversion_rate"]}

    _noise_reasonable_price = {1: settings["c1_noise_reasonable_price"], 2: settings["c2_noise_reasonable_price"], 3: settings["c3_noise_reasonable_price"]}
    _noise_elasticity = {1: settings["c1_noise_elasticity"], 2: settings["c2_noise_elasticity"], 3: settings["c3_noise_elasticity"]}
    _noise_max_conversion_rate = {1: settings["c1_noise_max_conversion_rate"], 2: settings["c2_noise_max_conversion_rate"], 3: settings["c3_noise_max_conversion_rate"]}

def generate_payment_characteristics(belonging_class):
    charachteristics = np.array([Payment_characteristics_generator._reasonable_price[belonging_class] + np.random.normal(0, Payment_characteristics_generator._noise_reasonable_price[belonging_class]),
                        Payment_characteristics_generator._elasticity[belonging_class] + np.random.normal(0, Payment_characteristics_generator._noise_elasticity[belonging_class]),
                        Payment_characteristics_generator._skepticism[belonging_class] + np.random.normal(0, Payment_characteristics_generator._noise_elasticity[belonging_class]),
                        Payment_characteristics_generator._max_conversion_rate[belonging_class] + np.random.normal(0, Payment_characteristics_generator._noise_max_conversion_rate[belonging_class])])
    # make all values positive
    charachteristics = np.abs(charachteristics)
    # make sure that the max conversion rate is not greater than 1
    charachteristics[3] = min(charachteristics[3], 1)
    return charachteristics

def generate_payment_characteristics_for_population(population):
    return np.array([generate_payment_characteristics(x) for x in population])

    

In [6]:
#generate pandas dataframe with f1,f2, class, payment characteristics
def generate_dataframe():
    population = generate_population()
    classes = classify(population)
    payment_characteristics = generate_payment_characteristics_for_population(classes)
    return pd.DataFrame(np.hstack((population, classes.reshape(-1,1), payment_characteristics)), columns=["f1", "f2", "class", "reasonable_price", "elasticity", "skepticism", "max_conversion_rate"])

population = generate_dataframe()
population.head()


,f1,f2,class,reasonable_price,elasticity,skepticism,max_conversion_rate
0,1.0,0.0,3.0,40.0,0.10,0.008,0.50
1,1.0,0.0,3.0,40.0,0.10,0.008,0.50
2,0.0,1.0,2.0,50.0,0.05,0.050,0.85
3,0.0,1.0,2.0,50.0,0.05,0.050,0.85
4,1.0,0.0,3.0,40.0,0.10,0.008,0.50


In [7]:
# For every user class, consider 5 different possible prices and use a Bernoulli distribution for every price. This specifies whether the user buys or not the item at that specific price. A sample of the Bernoulli must be independently drawn for every user who landed on the e-commerce website.
print(settings['prices'])
def generate_conversion_rate_per_price(
    preferred_price,
    elasticity,
    skepticism,
    max_conversion_rate,
    prices = settings['prices'],
    fix_seed = True,
    seed = 42,
    noise_level = settings["default_noise"]
    ):
    
    if fix_seed:
        np.random.seed(seed)
    
    rates = np.zeros_like(prices)

    above_preferred = prices >= preferred_price
    below_preferred = prices < preferred_price

    diff_above = prices[above_preferred] - preferred_price
    diff_below = preferred_price - prices[below_preferred]

    rates[above_preferred] = np.exp(-elasticity * diff_above)
    rates[below_preferred] = np.exp(-skepticism * diff_below)

    noise = np.random.normal(0, noise_level, prices.shape)
    rates += noise

    # Apply cap and ensure values are within [0, 1] range
    rates = np.clip(rates, 0, 1)

    return rates


#generate conversion rates per price for every user in the population
def generate_conversion_rates(population):
    conversion_rates = np.zeros((population.shape[0], settings["prices"].shape[0]))
    for i, row in population.iterrows():
        conversion_rates[i] = generate_conversion_rate_per_price(row["reasonable_price"], row["elasticity"], row["skepticism"], row["max_conversion_rate"])
    return conversion_rates

#attach conversion rates to the population dataframe, make one column for every price, name is cvr_{idx}
def attach_conversion_rates(population):
    conversion_rates = generate_conversion_rates(population)
    #return pd.concat([population, pd.DataFrame(conversion_rates, columns=settings["prices"])], axis=1)
    columns = ["cvr_{}".format(i) for i in range(settings["prices"].shape[0])]
    return pd.concat([population, pd.DataFrame(conversion_rates, columns=columns)], axis=1)

population = attach_conversion_rates(population)
population.head()

[ 10.  20.  30.  60. 100.]


,f1,f2,class,reasonable_price,elasticity,skepticism,max_conversion_rate,cvr_0,cvr_1,cvr_2,cvr_3,cvr_4
0,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479
1,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479
2,0.0,1.0,2.0,50.0,0.05,0.050,0.85,0.135335,0.223130,0.367879,0.606531,0.082085
3,0.0,1.0,2.0,50.0,0.05,0.050,0.85,0.135335,0.223130,0.367879,0.606531,0.082085
4,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479


In [8]:
#decide whether the user buys the item or not for every price
def generate_purchases(population):
    columns = ["cvr_{}".format(i) for i in range(settings["prices"].shape[0])]
    purchases = np.zeros((population.shape[0], settings["prices"].shape[0]))
    for i, row in population.iterrows():
        purchases[i] = np.random.binomial(1, row[columns])
    return purchases

#attach purchases to the population dataframe, make one column for every price
def attach_purchases(population):
    purchases = generate_purchases(population)
    columns = ["p{}".format(i) for i in range(settings["prices"].shape[0])]
    return pd.concat([population, pd.DataFrame(purchases, columns=columns)], axis=1)

population = attach_purchases(population)
population.head(15)


,f1,f2,class,reasonable_price,elasticity,skepticism,max_conversion_rate,cvr_0,cvr_1,cvr_2,cvr_3,cvr_4,p0,p1,p2,p3,p4
0,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,0.0,1.0,1.0,0.0,0.0
2,0.0,1.0,2.0,50.0,0.05,0.050,0.85,0.135335,0.223130,0.367879,0.606531,0.082085,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,2.0,50.0,0.05,0.050,0.85,0.135335,0.223130,0.367879,0.606531,0.082085,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,1.0,1.0,1.0,0.0,0.0
5,1.0,1.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,1.0,1.0,0.0,1.0,0.0
6,1.0,0.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,1.0,1.0,1.0,0.0,0.0
7,1.0,1.0,3.0,40.0,0.10,0.008,0.50,0.786628,0.852144,0.923116,0.135335,0.002479,1.0,1.0,1.0,0.0,0.0
8,0.0,0.0,1.0,30.0,0.10,0.010,0.75,0.818731,0.904837,1.000000,0.049787,0.000912,1.0,1.0,1.0,0.0,0.0
9,0.0,0.0,1.0,30.0,0.10,0.010,0.75,0.818731,0.904837,1.000000,0.049787,0.000912,1.0,0.0,1.0,0.0,0.0


In [9]:
#dump into csv
population.to_csv("population.csv")